In [ ]:
# imports
import requests
import json
from dateutil import parser
import pandas as pd
# clearly see columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
import time
import math
import matplotlib.pyplot as plt
import pickle
import seaborn as sb
from collections import Counter
import numpy as np
import time
from IPython.display import clear_output

# use nba_api to fetch data
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType

In [ ]:
season_list = ['2020-21', '2021-22', '2022-23']
# season_list = ['2010-11', '2011-12', '2012-13', '2013-14', '2014-15', '2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23']

In [ ]:
nba_teams = teams.get_teams()
nba_teams_id = [team['id'] for team in nba_teams]
nba_teams_tri= [team['abbreviation'] for team in nba_teams]
nba_teams_name= [team['full_name'] for team in nba_teams]

master_game_dict = dict()
master_game_list = list()

for year in season_list:
    for team in nba_teams_id:
        print("Team: {}, Season: {}".format(nba_teams_tri[nba_teams_id.index(team)], year))
#         create games dataframe
        temp_dp = leaguegamefinder.LeagueGameFinder(team_id_nullable=team, 
                                                    season_nullable=year,
                                                    season_type_nullable=SeasonType.regular).get_data_frames()[0]
        temp_dp.sort_values(by='GAME_ID', inplace=True)
        temp_dp.reset_index(drop=True, inplace=True)
        
        master_game_list.append(temp_dp)
        
        if nba_teams_tri[nba_teams_id.index(team)] in master_game_dict:
            master_game_dict[nba_teams_tri[nba_teams_id.index(team)]].append(temp_dp)
            
        else:
            master_game_dict[nba_teams_tri[nba_teams_id.index(team)]] = [temp_dp]
        
        time.sleep(1.5)
        clear_output(wait=True)

Team: CHA, Season: 2022-23


In [ ]:
 master_game_dict['CHA'][0]

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22020,1610612766,CHA,Charlotte Hornets,0022000010,2020-12-23,CHA @ CLE,L,240,114,45,90,0.500,16,44,0.364,8,15,0.533,8,24,32,29,10,4,15,17,-7.0
1,22020,1610612766,CHA,Charlotte Hornets,0022000022,2020-12-26,CHA vs. OKC,L,239,107,37,93,0.398,13,32,0.406,20,32,0.625,13,34,47,27,11,8,13,22,-2.0
2,22020,1610612766,CHA,Charlotte Hornets,0022000032,2020-12-27,CHA vs. BKN,W,241,106,43,96,0.448,10,34,0.294,10,17,0.588,13,39,52,35,8,3,12,25,2.0
3,22020,1610612766,CHA,Charlotte Hornets,0022000059,2020-12-30,CHA @ DAL,W,240,118,43,84,0.512,15,34,0.441,17,21,0.810,12,38,50,30,6,6,23,28,19.0
4,22020,1610612766,CHA,Charlotte Hornets,0022000069,2021-01-01,CHA vs. MEM,L,241,93,35,86,0.407,9,31,0.290,14,21,0.667,10,32,42,23,9,4,18,15,-15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,22020,1610612766,CHA,Charlotte Hornets,0022001004,2021-05-07,CHA vs. ORL,W,241,122,44,89,0.494,9,33,0.273,25,33,0.758,11,40,51,28,5,4,11,23,10.0
68,22020,1610612766,CHA,Charlotte Hornets,0022001020,2021-05-09,CHA vs. NOP,L,240,110,42,95,0.442,13,37,0.351,13,18,0.722,11,35,46,22,6,8,17,19,-2.0
69,22020,1610612766,CHA,Charlotte Hornets,0022001047,2021-05-13,CHA vs. LAC,L,239,90,33,84,0.393,9,33,0.273,15,19,0.789,8,27,35,24,9,1,10,15,-23.0
70,22020,1610612766,CHA,Charlotte Hornets,0022001064,2021-05-15,CHA @ NYK,L,265,109,39,102,0.382,13,49,0.265,18,24,0.750,15,36,51,28,5,2,8,18,-9.0


In [ ]:
# function to fetch first game day of every season in a list
def get_first_match(game_dict, season_list, nba_teams_tri):
    first_game_list = list()
    for season_index in range(len(season_list)):
        temp_list = list()
        for team in nba_teams_tri:
            temp_list.append(game_dict[team][season_index].loc[0]['GAME_DATE'])
        first_game_list.append(min(temp_list))
    return first_game_list
first_game_list = get_first_match(master_game_dict, season_list, nba_teams_tri)

In [ ]:
# craete dict entry for each team, give initial ELO and season start date
teamDict = dict()
for entry in nba_teams_tri:
    tempDict = dict()
    tempDict['tri'] = entry
    tempDict['ELO'] = 1500 
    tempDict['History'] = [{'date': parser.parse(master_game_dict[entry][0].loc[0]['GAME_DATE']).date(), 'ELO': tempDict['ELO']}]
    teamDict[entry] = tempDict

In [ ]:
# build intial dataframe detailing all games, split by team
master_games_df = pd.concat(master_game_list)
master_games_df.sort_values(by='GAME_ID', inplace=True)
master_games_df.reset_index(drop=True, inplace=True)
master_games_df

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22020,1610612744,GSW,Golden State Warriors,0022000001,2020-12-22,GSW @ BKN,L,240,99,37,99,0.374,10,33,0.303,15,23,0.652,13,34,47,26,6,6,18,24,-26.0
1,22020,1610612751,BKN,Brooklyn Nets,0022000001,2020-12-22,BKN vs. GSW,W,242,125,42,92,0.457,15,35,0.429,26,32,0.813,13,44,57,24,11,7,20,22,26.0
2,22020,1610612747,LAL,Los Angeles Lakers,0022000002,2020-12-22,LAL vs. LAC,L,240,109,38,81,0.469,9,29,0.310,24,31,0.774,8,37,45,22,4,2,19,20,-7.0
3,22020,1610612746,LAC,LA Clippers,0022000002,2020-12-22,LAC @ LAL,W,241,116,44,93,0.473,14,40,0.350,14,19,0.737,11,29,40,22,10,3,16,29,7.0
4,22020,1610612738,BOS,Boston Celtics,0022000003,2020-12-23,BOS vs. MIL,W,240,122,48,101,0.475,18,40,0.450,8,11,0.727,10,27,37,23,8,6,6,17,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4789,22022,1610612737,ATL,Atlanta Hawks,0022200085,2022-10-29,ATL @ MIL,L,239,115,45,94,0.479,13,29,0.448,12,15,0.800,12,30,42,19,5,2,12,23,-5.0
4790,22022,1610612742,DAL,Dallas Mavericks,0022200086,2022-10-29,DAL vs. OKC,L,264,111,38,95,0.400,8,40,0.200,27,34,0.794,12,37,49,17,8,5,16,30,-6.0
4791,22022,1610612760,OKC,Oklahoma City Thunder,0022200086,2022-10-29,OKC @ DAL,W,265,117,44,89,0.494,5,25,0.200,24,31,0.774,9,38,47,25,9,6,15,29,6.0
4792,22022,1610612763,MEM,Memphis Grizzlies,0022200087,2022-10-29,MEM @ UTA,L,240,123,50,88,0.568,12,27,0.444,11,15,0.733,10,30,40,24,11,5,20,26,-1.0


In [ ]:
# api to fetch boxscores by game id
from nba_api.live.nba.endpoints import boxscore

# list that contains all game stats 
master_games_stats_list = list()

# list of all games by their id,. sorted 
game_id_list = set([master_games_df['GAME_ID'] for game in master_games_df][0])
game_id_list = list(game_id_list)
game_id_list.sort()

# index used to detect change in season
season_index = 0

# loop to calculate team ELO per game / saeson
for game_id in game_id_list:
    
    box = boxscore.BoxScore(game_id) 
    
    if box.get_dict()['meta']['code'] == 200:
        # game id
        game_id = box.get_dict()['game']['gameId']
        # game date
        game_date = parser.parse(box.get_dict()['game']['gameTimeUTC']).date()
        # team code
        home_tri = box.get_dict()['game']['homeTeam']['teamTricode']
        away_tri = box.get_dict()['game']['awayTeam']['teamTricode']

        print('game id: {} \t home team: {} \t away team: {}'.format(game_id, home_tri, away_tri))

        # team stats
        box_home = box.get_dict()['game']['homeTeam']['statistics']
        box_away = box.get_dict()['game']['awayTeam']['statistics']

        if game_date >= parser.parse(first_game_list[season_index]).date(): 
            if season_index < (len(first_game_list) - 1):
                if game_date >= parser.parse(first_game_list[season_index + 1]).date():
                    print('Season Change')
                    # season change
                    season_index = season_index + 1
                    for teamKey in list(teamDict.keys()):
                        teamDict[teamKey]['ELO'] = (teamDict[teamKey]['ELO'] * 0.75) + (0.25 * 1505)

            if box_home['points'] > box_away['points']: # home team win 
                print('home win', '\n')
            # Winner  ELO
                # S Team
                Steam = 1
                # E Team
                Eteam = 1 / (1 + (pow(10, ((teamDict[away_tri]['ELO'] - teamDict[home_tri]['ELO']) / 400))))
                # Ri
                Ri = teamDict[home_tri]['ELO']
                # k
                MOV = int(box_home['points']) - int(box_away['points'])
                ################## Error Maybe Here ##################
                # ELO_diff
                ELOdiff = teamDict[home_tri]['ELO'] + 100 - teamDict[away_tri]['ELO']
                k = 20 * (pow((MOV + 3),0.8) / (7.5 + (0.006 * (ELOdiff))))
                ######################################################
                # Ri+1
                WinnerRii = Ri + (k * (Steam - Eteam))

            # Loser ELO
                # S Team
                Steam = 0
                # E Team
                Eteam = 1 / (1 + (pow(10, ((teamDict[home_tri]['ELO'] - teamDict[away_tri]['ELO']) / 400))))
                # Ri
                Ri = teamDict[away_tri]['ELO']
                # Ri+1
                LoserRii = Ri + (k * (Steam - Eteam))

                print('Home ELO: {} \t Away ELO: {}'.format(WinnerRii, LoserRii))

            # Setting new ELO   
                box_home['team_elo'] = teamDict[home_tri]['ELO']
                box_home['opp_elo'] = teamDict[away_tri]['ELO']
                box_away['team_elo'] = teamDict[away_tri]['ELO']
                box_away['opp_elo'] = teamDict[home_tri]['ELO']

            # Winner 
                box_home['game_id'] = game_id
                box_home['game_date'] = game_date 
                box_home['home'] = 1
                box_home['team_name'] = home_tri   
                box_home['opp_name'] = away_tri   
                box_home['win'] = 1
                teamDict[home_tri]['ELO'] = math.floor(WinnerRii)
            # Loser
                box_away['game_id'] = game_id
                box_away['game_date'] = game_date 
                box_away['team_name'] = away_tri  
                box_away['opp_name'] = home_tri
                box_away['home'] = 0
                box_away['win'] = 0
                teamDict[away_tri]['ELO'] = math.floor(LoserRii)

            elif box_home['points'] < box_away['points']: # home lose
                print('away win', '\n')

            # Winner  ELO
                # S Team
                Steam = 1
                # E Team
                Eteam = 1 / (1 + (pow(10, ((teamDict[home_tri]['ELO'] - teamDict[away_tri]['ELO']) / 400))))
                # Ri
                Ri = teamDict[away_tri]['ELO']
                # k
                MOV = int(box_away['points']) - int(box_home['points'])
                ################## Error Maybe Here ##################
                # ELO_diff
                ELOdiff = (teamDict[home_tri]['ELO'] + 100 - teamDict[away_tri]['ELO']) * (-1)
                k = 20 * (pow((MOV + 3),0.8) / (7.5 + (0.006 * (ELOdiff))))
                ######################################################
                # Ri+1
                WinnerRii = Ri + (k * (Steam - Eteam))
            # Loser ELO
                # S Team
                Steam = 0
                # E Team
                Eteam = 1 / (1 + (pow(10, ((teamDict[away_tri]['ELO'] - teamDict[home_tri]['ELO']) / 400))))
                # Ri
                Ri = teamDict[home_tri]['ELO'] ## CHECK THIS 
                # Ri+1
                LoserRii = Ri + (k * (Steam - Eteam))

                print('Home ELO: {} \t Away ELO: {}'.format(WinnerRii, LoserRii))

            # Setting new ELO
                box_away['team_elo'] = teamDict[away_tri]['ELO']
                box_away['opp_elo'] = teamDict[home_tri]['ELO']
                box_home['team_elo'] = teamDict[home_tri]['ELO']
                box_home['opp_elo'] = teamDict[away_tri]['ELO']

            # Winner 
                box_away['game_id'] = game_id 
                box_away['game_date'] = game_date 
                box_away['team_name'] = away_tri  
                box_away['opp_name'] = home_tri
                box_away['home'] = 0       
                box_away['win'] = 1
                teamDict[away_tri]['ELO'] = math.floor(WinnerRii)
            # Loser
                box_home['game_id'] = game_id
                box_home['game_date'] = game_date 
                box_home['team_name'] = home_tri   
                box_home['opp_name'] = away_tri  
                box_home['home'] = 1
                box_home['win'] = 0 
                teamDict[home_tri]['ELO'] = math.floor(LoserRii)        

        # Setting Historical ELO
            teamDict[home_tri]['History'].append({'date': game_date, 
                                                  'ELO': box_home['team_elo']})
            teamDict[away_tri]['History'].append({'date': game_date, 
                                                  'ELO': box_away['team_elo']})
            master_games_stats_list.append(box_home)
            master_games_stats_list.append(box_away)

            time.sleep(1)
            clear_output(wait=True)

game id: 0022001063 	 home team: MIN 	 away team: BOS
away win 

Home ELO: 1480.1506326234478 	 Away ELO: 1379.8493673765522


In [ ]:
# dataframe that contains all game stats and team ELO
df = pd.DataFrame(master_games_stats_list)
df.sort_values(by=(['game_date', 'game_id']), inplace=True)
# df=df.dropna().reset_index(drop=True)
df

In [ ]:
# create season feature
def create_season(row):
    if '00220' in row['game_id']:
        return '2020'
    if '00221' in row['game_id']:
        return '2021'
    if '00222' in row['game_id']:
        return '2022'
df['season'] = df.apply (lambda row: create_season(row), axis=1)

In [ ]:
df

In [ ]:
del df['biggestLeadScore']
del df['biggestScoringRunScore']
del df['minutes']
del df['minutesCalculated']
del df['timeLeading']

In [ ]:
# load models
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

# create model that predicts games win / loss based on current game performance
# Test / Train Split
features = df.drop(columns = ['game_id', 'game_date', 
                              'team_name', 'opp_name', 'win'])
label = df['win']
x_train, x_test, y_train, y_test = train_test_split(features, label, test_size = 0.4)
# create complex Logistic Regression  
log_model = LogisticRegression(max_iter=2000, verbose=2, random_state=42)
log_model.fit(x_train, y_train)
y_pred_log = log_model.predict(x_test)

clear_output(wait=True)

print('Logistic Regression: ' + '\t' + str(metrics.accuracy_score(y_test, y_pred_log)))
rf = RandomForestClassifier(random_state=42)
rf.fit(x_train, y_train)
y_pred_rf = rf.predict(x_test)
print('Random Forest: ' + '\t\t' + str(metrics.accuracy_score(y_test, y_pred_rf)))

In [ ]:
seasonAllStatSplitPred = df.to_dict('records')
seasonAllStatSplitPredList = list()
for game_one in seasonAllStatSplitPred:
    teamPairID_one = [game_one['team_name'], game_one['opp_name']]
    teamPair = [game_one]
    flag = 0
    for game_two in seasonAllStatSplitPred:
        teamPairID_two = [game_two['team_name'], game_two['opp_name']]
        if (teamPairID_one == teamPairID_two) and (flag == 0) and (teamPair[0]['home'] == game_two['home']):
            flag = 1
        elif (teamPairID_one == teamPairID_two) and (flag == 1) and (teamPair[0]['home'] == game_two['home']):
            teamPair.append(game_two)
            tempDict = teamPair[0].copy()
            tempDict['present_team_elo'] = teamPair[-1]['team_elo']
            tempDict['present_opp_elo'] = teamPair[-1]['opp_elo']
            tempDict['present_home_court'] = teamPair[-1]['home']
            tempDict['present_win'] = teamPair[-1]['win']
            del teamPair[0]
            seasonAllStatSplitPredList.append(tempDict)   

In [ ]:
seasonAllStatSplitPred = pd.DataFrame(seasonAllStatSplitPredList)
# Sort by Date -> Game ID
seasonAllStatSplitPred.sort_values(by=(['game_date', 'game_id']), inplace=True)
# nan_value = float("NaN")
# seasonAllStatSplitPred.replace("", nan_value, inplace=True)
# seasonAllStatSplitPred=seasonAllStatSplitPred.dropna()
# seasonAllStatSplitPred=seasonAllStatSplitPred.dropna(axis=0)
# seasonAllStatSplitPred=seasonAllStatSplitPred.dropna().reset_index(drop=True)
seasonAllStatSplitPred

In [ ]:
####################################################################################################################################

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier

rr = RidgeClassifier(alpha = 1)
split = TimeSeriesSplit(n_splits=3)
sfs = SequentialFeatureSelector(rr, n_features_to_select = 30, direction="forward",cv=split)

removed_columns = ['game_id', 'game_date', 'team_name', 'opp_name', 'win', 'season', 'present_win']
selected_columns = seasonAllStatSplitPred.columns[~seasonAllStatSplitPred.columns.isin(removed_columns)]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
seasonAllStatSplitPred[selected_columns] = scaler.fit_transform(seasonAllStatSplitPred[selected_columns])

seasonAllStatSplitPred

In [ ]:
sfs.fit(seasonAllStatSplitPred[selected_columns], seasonAllStatSplitPred['present_win'])

In [ ]:
predictors = list(selected_columns[sfs.get_support()])
print(predictors)

In [ ]:
def backtest(data, model, predictors, start=1, step=1):
    all_predictions = []
    
    seasons = sorted(data['season'].unique())
    
    for i in range(start, len(seasons), step):
        season = seasons[i]
        
        train = data[data['season'] < season]
        test = data[data['season'] == season]
        
        model.fit(train[predictors], train['present_win'])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        
        combined = pd.concat((test['present_win'], preds), axis = 1)
        combined.columns = ['actual', 'predictions']
        
        all_predictions.append(combined)
        
    return pd.concat(all_predictions)

In [ ]:
predictions = backtest(seasonAllStatSplitPred, rr, predictors)

In [ ]:
predictions

In [ ]:
####################################################################################################################################

In [ ]:
from sklearn.linear_model import RidgeClassifier

rr = RidgeClassifier(alpha = 1)

# create model that predicts games win / loss based on previous game (n = 1) performance
# Test / Train Split
features = seasonAllStatSplitPred.drop(columns = ['game_id', 'game_date', 'team_name', 'opp_name', 'present_win'])

label = seasonAllStatSplitPred['present_win']
x_train, x_test, y_train, y_test = train_test_split(features, label, test_size = 0.3)
# create complex Logistic Regression  
log_model = LogisticRegression(max_iter=2000, verbose=2, random_state=42)
log_model.fit(x_train, y_train)
y_pred_log = log_model.predict(x_test)

clear_output(wait=True)

##################
rr.fit(x_train, y_train)
y_pred_rr = rr.predict(x_test)
print('Ridge Classifier: ' + '\t' + str(metrics.accuracy_score(y_test, y_pred_rr)))
####################

print('Logistic Regression: ' + '\t' + str(metrics.accuracy_score(y_test, y_pred_log)))
rf = RandomForestClassifier(random_state=42)
rf.fit(x_train, y_train)
y_pred_rf = rf.predict(x_test)
print('Random Forest: ' + '\t\t' + str(metrics.accuracy_score(y_test, y_pred_rf)))

In [ ]:
# nba_teams_name.sort()
# nba_teams_name

In [ ]:
# Test / Train Split
def randomForest(features_in, label_in, team_home, team_away):
    home_win_avg_list = list()
    away_win_avg_list = list()

    for i in range(10):  

        x_train, x_test, y_train, y_test = train_test_split(features_in, label_in, test_size = 0.3)

        rf = RandomForestClassifier(random_state=42)
        rf.fit(x_train, y_train)
        # y_pred_rf = rf.predict(x_test)

        # log_model = LogisticRegression(max_iter=20000, verbose=2, random_state=42)
        # log_model.fit(x_train, y_train)
        # y_pred_log = log_model.predict(x_test)

        testTestTest = pd.DataFrame([team_home, team_away])
        testTestTestTest = testTestTest.drop(columns = ['game_id', 'game_date', 'team_name', 'opp_name', 'present_win'])
        # make a prediction
        ynew = rf.predict(testTestTestTest)
        # ynew = log_model.predict(testTestTestTest)

        # print("Predicted Home Win = %s" % ynew[0])
        home_win_avg_list.append(ynew[0])
        # print("Predicted Away Win = %s" % ynew[1])
        away_win_avg_list.append(ynew[1])

    print("Final predicted win: ")
    print("Predicted %s Home Win = %s" % (home_name, max(set(home_win_avg_list), key = home_win_avg_list.count)))
    print("Confidence: {}".format(((pd.Series(home_win_avg_list).value_counts()) / (len(home_win_avg_list)) * 100)))
    print("Predicted %s Away Win = %s" % (away_name, max(set(away_win_avg_list), key = away_win_avg_list.count)))
    print("Confidence: {}".format(((pd.Series(away_win_avg_list).value_counts()) / (len(away_win_avg_list)) * 100)))

In [ ]:
# Test / Train Split
def ridgeClassifier(features_in, label_in, team_home, team_away):
    home_win_avg_list = list()
    away_win_avg_list = list()

    for i in range(10):  

        x_train, x_test, y_train, y_test = train_test_split(features_in, label_in, test_size = 0.3)

        rr = RidgeClassifier(alpha = 1)
        rr.fit(x_train, y_train)
        # y_pred_rf = rf.predict(x_test)

        # log_model = LogisticRegression(max_iter=20000, verbose=2, random_state=42)
        # log_model.fit(x_train, y_train)
        # y_pred_log = log_model.predict(x_test)

        testTestTest = pd.DataFrame([team_home, team_away])
        testTestTestTest = testTestTest.drop(columns = ['game_id', 'game_date', 'team_name', 'opp_name', 'present_win'])
        # make a prediction
        ynew = rr.predict(testTestTestTest)
        # ynew = log_model.predict(testTestTestTest)

        # print("Predicted Home Win = %s" % ynew[0])
        home_win_avg_list.append(ynew[0])
        # print("Predicted Away Win = %s" % ynew[1])
        away_win_avg_list.append(ynew[1])

    print("Final predicted win: ")
    print("Predicted %s Home Win = %s" % (home_name, max(set(home_win_avg_list), key = home_win_avg_list.count)))
    print("Confidence: {}".format(((pd.Series(home_win_avg_list).value_counts()) / (len(home_win_avg_list)) * 100)))
    print("Predicted %s Away Win = %s" % (away_name, max(set(away_win_avg_list), key = away_win_avg_list.count)))
    print("Confidence: {}".format(((pd.Series(away_win_avg_list).value_counts()) / (len(away_win_avg_list)) * 100)))

In [ ]:
# Test / Train Split
def logRegression(features_in, label_in, team_home, team_away):
    home_win_avg_list = list()
    away_win_avg_list = list()

    for i in range(10):  

        x_train, x_test, y_train, y_test = train_test_split(features_in, label_in, test_size = 0.3)

        # rf = RandomForestClassifier(random_state=42)
        # rf.fit(x_train, y_train)
        # y_pred_rf = rf.predict(x_test)

        log_model = LogisticRegression(max_iter=20000, verbose=2, random_state=42)
        log_model.fit(x_train, y_train)
        # y_pred_log = log_model.predict(x_test)

        testTestTest = pd.DataFrame([team_home, team_away])
        testTestTestTest = testTestTest.drop(columns = ['game_id', 'game_date', 'team_name', 'opp_name', 'present_win'])
        # make a prediction
        # ynew = rf.predict(testTestTestTest)
        ynew = log_model.predict(testTestTestTest)
        # print("Predicted Home Win = %s" % ynew[0])
        home_win_avg_list.append(ynew[0])
        # print("Predicted Away Win = %s" % ynew[1])
        away_win_avg_list.append(ynew[1])

        # clear_output(wait=True)

    print("Final predicted win: ")
    print("Predicted %s Home Win = %s" % (home_name, max(set(home_win_avg_list), key = home_win_avg_list.count)))
    print("Confidence: {}".format(((pd.Series(home_win_avg_list).value_counts()) / (len(home_win_avg_list)) * 100)))
    print("Predicted %s Away Win = %s" % (away_name, max(set(away_win_avg_list), key = away_win_avg_list.count)))
    print("Confidence: {}".format(((pd.Series(away_win_avg_list).value_counts()) / (len(away_win_avg_list)) * 100)))

In [ ]:
# Query nba.live.endpoints.scoreboard and  list games in localTimeZone
from datetime import datetime, timezone
from dateutil import parser
from nba_api.live.nba.endpoints import scoreboard

f = "{gameId}: {homeTeam} vs. {awayTeam}" 

game_pair = list()

board = scoreboard.ScoreBoard()
print("ScoreBoardDate: " + board.score_board_date)
games = board.games.get_dict()
for game in games:
    gameTimeLTZ = parser.parse(game["gameTimeUTC"]).replace(tzinfo=timezone.utc).astimezone(tz=None)
    print(f.format(gameId=game['gameId'], 
                   awayTeam=(game['awayTeam']['teamCity']) + ' ' + (game['awayTeam']['teamName']), 
                   homeTeam=(game['homeTeam']['teamCity']) + ' ' + (game['homeTeam']['teamName'])))
    game_pair.append(((game['homeTeam']['teamCity']) + ' ' + (game['homeTeam']['teamName']), (game['awayTeam']['teamCity']) + ' ' + (game['awayTeam']['teamName'])))

In [28]:
seasonAllStatSplitPredDict = seasonAllStatSplitPred.to_dict('records')
# seasonAllStatSplitPredDict.reverse()

for game in game_pair:

    home_name = game[0]
    away_name = game[1]

    Home_ID = nba_teams_tri[nba_teams_name.index(home_name)]
    Away_ID = nba_teams_tri[nba_teams_name.index(away_name)]

    gamePairList_one = list()
    gamePairList_one_elo = list()
    gamePairList_Two = list()
    gamePairList_Two_elo = list()
    for game in seasonAllStatSplitPredDict:
        if game['team_name'] == Home_ID and game['opp_name'] == Away_ID:
            gamePairList_one.append(game)
        if game['team_name'] == Away_ID and game['opp_name'] == Home_ID:
            gamePairList_Two.append(game)
        if game['team_name'] == Home_ID:
            gamePairList_one_elo.append(game)
        if game['team_name'] == Away_ID:
            gamePairList_Two_elo.append(game)
    #print('Last Face Off Bucks Home:')
    #print(gamePairList_one[1])
    #print(gamePairList_Two[1])
    #print('Last ELO Home:')
    #print(gamePairList_one_elo[0])
    #print('Last ELO Away:')
    #print(gamePairList_Two_elo[0])

    teamHome = gamePairList_one[1].copy()
    teamHome['present_team_elo'] = (gamePairList_one_elo[0]['team_elo'] * 0.75) + (0.25 * 1505)
    teamHome['present_opp_elo'] = (gamePairList_Two_elo[0]['opp_elo'] * 0.75) + (0.25 * 1505)
    teamHome['present_home_court'] = 1
    teamAway = gamePairList_Two[1].copy() 
    teamAway['present_team_elo'] = (gamePairList_Two_elo[0]['team_elo'] * 0.75) + (0.25 * 1505)
    teamAway['present_opp_elo'] = (gamePairList_one_elo[0]['opp_elo'] * 0.75) + (0.25 * 1505)
    teamAway['present_home_court'] = 0
    
    print('###################################################################################')
    randomForest(features, label, teamHome, teamAway)
    print()
    ridgeClassifier(features, label, teamHome, teamAway)
    print()
    logRegression(features, label, teamHome, teamAway)
    print()
    print('###################################################################################')

###################################################################################
Final predicted win: 
Predicted Sacramento Kings Home Win = 1
Confidence: 1    80.0
0    20.0
dtype: float64
Predicted Miami Heat Away Win = 0
Confidence: 0    80.0
1    20.0
dtype: float64

Final predicted win: 
Predicted Sacramento Kings Home Win = 0
Confidence: 0    70.0
1    30.0
dtype: float64
Predicted Miami Heat Away Win = 1
Confidence: 1    100.0
dtype: float64



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent work

Final predicted win: 
Predicted Sacramento Kings Home Win = 0
Confidence: 0    100.0
dtype: float64
Predicted Miami Heat Away Win = 1
Confidence: 1    100.0
dtype: float64

###################################################################################
###################################################################################
Final predicted win: 
Predicted Charlotte Hornets Home Win = 0
Confidence: 0    90.0
1    10.0
dtype: float64
Predicted Golden State Warriors Away Win = 1
Confidence: 1    80.0
0    20.0
dtype: float64

Final predicted win: 
Predicted Charlotte Hornets Home Win = 0
Confidence: 0    100.0
dtype: float64
Predicted Golden State Warriors Away Win = 1
Confidence: 1    100.0
dtype: float64



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent work

Final predicted win: 
Predicted Charlotte Hornets Home Win = 0
Confidence: 0    100.0
dtype: float64
Predicted Golden State Warriors Away Win = 1
Confidence: 1    100.0
dtype: float64

###################################################################################
###################################################################################
Final predicted win: 
Predicted Brooklyn Nets Home Win = 1
Confidence: 1    100.0
dtype: float64
Predicted Indiana Pacers Away Win = 0
Confidence: 0    100.0
dtype: float64

Final predicted win: 
Predicted Brooklyn Nets Home Win = 1
Confidence: 1    100.0
dtype: float64
Predicted Indiana Pacers Away Win = 0
Confidence: 0    100.0
dtype: float64



...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent work

Final predicted win: 
Predicted Dallas Mavericks Home Win = 0
Confidence: 0    100.0
dtype: float64
Predicted Oklahoma City Thunder Away Win = 1
Confidence: 1    100.0
dtype: float64

###################################################################################
###################################################################################
Final predicted win: 
Predicted Utah Jazz Home Win = 0
Confidence: 0    90.0
1    10.0
dtype: float64
Predicted Memphis Grizzlies Away Win = 1
Confidence: 1    90.0
0    10.0
dtype: float64

Final predicted win: 
Predicted Utah Jazz Home Win = 1
Confidence: 1    100.0
dtype: float64
Predicted Memphis Grizzlies Away Win = 0
Confidence: 0    100.0
dtype: float64



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent work

Final predicted win: 
Predicted Utah Jazz Home Win = 1
Confidence: 1    100.0
dtype: float64
Predicted Memphis Grizzlies Away Win = 0
Confidence: 0    100.0
dtype: float64

###################################################################################


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s finished
